In [4]:
# Import numpy as np
import pandas as pd

attribute_names = []
X = []
y = []
class_names = ['False', 'True']

# Load csv dataset (Brain Tumor.csv).
data = pd.read_csv(r'./data/Brain Tumor.csv', header=0)
dataframe = pd.DataFrame(data)

# Attribute_names label.
attribute_names = list(data.columns)

# All data features with 'Class' removed.
X = dataframe.drop(['Class', 'Image'], axis=1).values

# Classifier for brain tumor (true or false).
y = dataframe['Class'].to_numpy()

In [5]:
# Import splitting, decision tree, gridsearch
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Split data into training and test samples (70/30):
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Create model to classify
model = DecisionTreeClassifier()

# Define hyperparameters in dictionary
parameters = {
    'splitter' : ['best', 'random'],
    'criterion' : ['gini', 'entropy'],
    'max_features': ['log2', 'sqrt', 'auto'],
    'max_depth': [2, 4, 6, 8, 10, 15, 20],
    'min_samples_split': [2, 3, 5, 7, 9],
    'min_samples_leaf': [1,5,8,11],
    'random_state' : [0,1,2,3,4,5]
}

# Gridsearch object, with 5-cross validation
grid_search_dt = GridSearchCV(estimator = model, param_grid = parameters, scoring = 'accuracy', cv = 5, verbose = 1)

# Fit the data
grid_search_dt.fit(x_train, y_train)

Fitting 5 folds for each of 10080 candidates, totalling 50400 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 4, 6, 8, 10, 15, 20],
                         'max_features': ['log2', 'sqrt', 'auto'],
                         'min_samples_leaf': [1, 5, 8, 11],
                         'min_samples_split': [2, 3, 5, 7, 9],
                         'random_state': [0, 1, 2, 3, 4, 5],
                         'splitter': ['best', 'random']},
             scoring='accuracy', verbose=1)

In [6]:
print(grid_search_dt.best_estimator_)

DecisionTreeClassifier(criterion='entropy', max_depth=8, max_features='log2',
                       min_samples_split=3, random_state=0)


In [7]:
print(grid_search_dt.score(x_test, y_test))

0.9805137289636847


In [8]:
print(grid_search_dt.cv_results_)

{'mean_fit_time': array([0.00184813, 0.00073633, 0.00176177, ..., 0.00081711, 0.00355   ,
       0.00100026]), 'std_fit_time': array([1.12324592e-03, 2.34786240e-04, 1.95020720e-04, ...,
       2.38126500e-04, 4.87771234e-04, 9.36836372e-07]), 'mean_score_time': array([0.00011115, 0.0002492 , 0.00021062, ..., 0.00011106, 0.00061336,
       0.00040007]), 'std_score_time': array([0.0002223 , 0.00023101, 0.00025828, ..., 0.00022211, 0.00037144,
       0.00048998]), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', ..., 'entropy', 'entropy',
                   'entropy'],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[2, 2, 2, ..., 20, 20, 20],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_max_features': masked_array(data=['log2', 'log2', 'log2', ..., 'auto', 'auto', 'auto'],
             mask=[